In [1]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2022-01-19
# GitHub: https://github.com/jaaack-wang 

## Get TensorFlow

In case you have not installed TensorFlow, run the following cell.

In [2]:
# # Requires the latest pip
# !pip3 install --upgrade pip
# !pip3 install tensorflow

## Preprocess and numericalize text data

In [3]:
from utils import *
import jieba  # ---> tokenizer for Chinese

# ---- load dataset ----
train, dev, test = load_dataset(['train.tsv', 'dev.tsv', 'test.tsv'])

# ---- numericalize the train set ----
V = TextVectorizer(jieba.lcut) 
text = gather_text(train) # for collecting texts from train set
V.build_vocab(text) # for building mapping vocab_to_idx dictionary and text_encoder

train_encoded = list(encode_dataset(train, encoder=V)) # encodoing train set
dev_encoded = list(encode_dataset(dev, encoder=V)) # encodoing dev set for validation
test_encoded  = list(encode_dataset(test, encoder=V)) # encodoing dev set for prediction

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w9/d_nplhzj4qx35xxlgljgdtjh0000gn/T/jieba.cache
Loading model cost 0.705 seconds.
Prefix dict has been built successfully.


Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.


### A note

There are multiple ways to use tensorflow to train a model, but the easiest one is to employ the `fit` method. In this `fit` function, the `inputs` and `targets` (labels) from the train set should be separately provided, and the `inputs` and `targets` from the dev set should be put inside a list or tuple. And both the `inputs` and `targets` should not be batched, as there is another builtin parameter called `batch_size` that will create mini batches for us. Nevertheless, to maintain consistency, this tutorial decided to still use the `build_batches` function that we will build together later in other tutorials. This `build_batches` function will help normalize the text seq length, which tensorflow's `fit` method does not provide. 

A better way of using packages in the tensorflow ecosystem to preprocess and numericalize text data will be introduced separately, just as what I intended to do for the other two deep learning frameworks.

In [4]:
# ---- build mini batches for the train and dev set ----
train_batched = build_batches(train_encoded, batch_size=10000, 
                              max_seq_len=128, include_seq_len=False)

dev_batched = build_batches(dev_encoded, batch_size=10000, 
                            max_seq_len=128, include_seq_len=False)

test_batched = build_batches(test_encoded, batch_size=10000, 
                             max_seq_len=128, include_seq_len=False)

train_X, train_Y = train_batched[0]
dev_X, dev_Y = dev_batched[0]
test_X, test_Y = test_batched[0]

## Training and evaluating models

In [5]:
from tensorflow import keras

## BoW (Bag of Words)

In [6]:
from tf_models.BoW import BoW


model = BoW(len(V.vocab_to_idx), 1)
model.compile(optimizer=keras.optimizers.Adam(5e-4),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[["accuracy"]])

%time model.fit(train_X, train_Y, epochs=5, batch_size=64, validation_data=(dev_X, dev_Y))

2022-01-19 23:56:48.788658: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-19 23:56:49.143742: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
63/63 [==============================] - 2s 19ms/step - loss: 0.5528 - accuracy: 0.7318 - val_loss: 0.4173 - val_accuracy: 0.8417
Epoch 2/5
63/63 [==============================] - 1s 17ms/step - loss: 0.2875 - accuracy: 0.9160 - val_loss: 0.3348 - val_accuracy: 0.8750
Epoch 3/5
63/63 [==============================] - 1s 17ms/step - loss: 0.1703 - accuracy: 0.9555 - val_loss: 0.2959 - val_accuracy: 0.8783
Epoch 4/5
63/63 [==============================] - 1s 17ms/step - loss: 0.1116 - accuracy: 0.9753 - val_loss: 0.2815 - val_accuracy: 0.8800
Epoch 5/5
63/63 [==============================] - 1s 17ms/step - loss: 0.0819 - accuracy: 0.9837 - val_loss: 0.3094 - val_accuracy: 0.8725
CPU times: user 17.4 s, sys: 2.64 s, total: 20 s
Wall time: 6.3 s


In [7]:
model.evaluate(test_X, test_Y)

38/38 [==============================] - 0s 1ms/step - loss: 0.2872 - accuracy: 0.8700


[0.28718623518943787, 0.8700000047683716]

## CNN (Convolutional Neural Network)

In [8]:
from tf_models.CNN import CNN

In [9]:
model = CNN(len(V.vocab_to_idx), 1)
model.compile(optimizer=keras.optimizers.Adam(5e-4),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[["accuracy"]])

%time model.fit(train_X, train_Y, epochs=5, batch_size=64, validation_data=(dev_X, dev_Y))

Epoch 1/5
63/63 [==============================] - 3s 44ms/step - loss: 0.6816 - accuracy: 0.5950 - val_loss: 0.6487 - val_accuracy: 0.7708
Epoch 2/5
63/63 [==============================] - 3s 42ms/step - loss: 0.5471 - accuracy: 0.7840 - val_loss: 0.4776 - val_accuracy: 0.8125
Epoch 3/5
63/63 [==============================] - 3s 42ms/step - loss: 0.3323 - accuracy: 0.8903 - val_loss: 0.3393 - val_accuracy: 0.8533
Epoch 4/5
63/63 [==============================] - 3s 42ms/step - loss: 0.1570 - accuracy: 0.9565 - val_loss: 0.2860 - val_accuracy: 0.8867
Epoch 5/5
63/63 [==============================] - 3s 43ms/step - loss: 0.0660 - accuracy: 0.9877 - val_loss: 0.2893 - val_accuracy: 0.8833
CPU times: user 49.6 s, sys: 14.5 s, total: 1min 4s
Wall time: 14 s


In [10]:
model.evaluate(test_X, test_Y)

38/38 [==============================] - 0s 4ms/step - loss: 0.2617 - accuracy: 0.8950


[0.2617317736148834, 0.8949999809265137]

## RNN (Recurrent neural network)

## SimpleRNN

In [11]:
from tf_models.S_RNN import SimpleRNN

In [12]:
model = SimpleRNN(len(V.vocab_to_idx), 1, bidirectional=False)
model.compile(optimizer=keras.optimizers.Adam(5e-4),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[["accuracy"]])

%time model.fit(train_X, train_Y, epochs=5, batch_size=64, validation_data=(dev_X, dev_Y))

Epoch 1/5
63/63 [==============================] - 5s 65ms/step - loss: 0.6819 - accuracy: 0.5598 - val_loss: 0.6753 - val_accuracy: 0.5842
Epoch 2/5
63/63 [==============================] - 4s 59ms/step - loss: 0.5229 - accuracy: 0.7610 - val_loss: 0.4445 - val_accuracy: 0.7967
Epoch 3/5
63/63 [==============================] - 4s 60ms/step - loss: 0.2644 - accuracy: 0.8953 - val_loss: 0.3497 - val_accuracy: 0.8517
Epoch 4/5
63/63 [==============================] - 4s 56ms/step - loss: 0.1012 - accuracy: 0.9695 - val_loss: 0.4123 - val_accuracy: 0.8492
Epoch 5/5
63/63 [==============================] - 4s 58ms/step - loss: 0.0534 - accuracy: 0.9885 - val_loss: 0.5034 - val_accuracy: 0.7883
CPU times: user 58.4 s, sys: 18 s, total: 1min 16s
Wall time: 19.9 s


In [13]:
model.evaluate(test_X, test_Y)

38/38 [==============================] - 0s 7ms/step - loss: 0.4580 - accuracy: 0.8050


[0.4580153822898865, 0.8050000071525574]

## LSTM (Long short-term memory)

In [14]:
from tf_models.LSTM import LSTM

In [15]:
model = LSTM(len(V.vocab_to_idx), 1, bidirectional=False)
model.compile(optimizer=keras.optimizers.Adam(5e-4),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[["accuracy"]])

%time model.fit(train_X, train_Y, epochs=5, batch_size=64, validation_data=(dev_X, dev_Y))

Epoch 1/5
63/63 [==============================] - 12s 147ms/step - loss: 0.6592 - accuracy: 0.6202 - val_loss: 0.5304 - val_accuracy: 0.8183
Epoch 2/5
63/63 [==============================] - 9s 137ms/step - loss: 0.4057 - accuracy: 0.8720 - val_loss: 0.3850 - val_accuracy: 0.8383
Epoch 3/5
63/63 [==============================] - 9s 136ms/step - loss: 0.2446 - accuracy: 0.9218 - val_loss: 0.3728 - val_accuracy: 0.8675
Epoch 4/5
63/63 [==============================] - 9s 136ms/step - loss: 0.1120 - accuracy: 0.9710 - val_loss: 0.3360 - val_accuracy: 0.8725
Epoch 5/5
63/63 [==============================] - 8s 133ms/step - loss: 0.0603 - accuracy: 0.9852 - val_loss: 0.6199 - val_accuracy: 0.8483
CPU times: user 2min 13s, sys: 40.7 s, total: 2min 54s
Wall time: 46.1 s


In [16]:
model.evaluate(test_X, test_Y)

38/38 [==============================] - 1s 19ms/step - loss: 0.5448 - accuracy: 0.8683


[0.544813334941864, 0.8683333396911621]

## GRU (Gated recurrent units) 

In [17]:
from tf_models.GRU import GRU

In [18]:
model = GRU(len(V.vocab_to_idx), 1, bidirectional=False)
model.compile(optimizer=keras.optimizers.Adam(5e-4),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[["accuracy"]])

%time model.fit(train_X, train_Y, epochs=5, batch_size=64, validation_data=(dev_X, dev_Y))

Epoch 1/5
63/63 [==============================] - 10s 122ms/step - loss: 0.6837 - accuracy: 0.5690 - val_loss: 0.6377 - val_accuracy: 0.6700
Epoch 2/5
63/63 [==============================] - 7s 114ms/step - loss: 0.4196 - accuracy: 0.8190 - val_loss: 0.3533 - val_accuracy: 0.8558
Epoch 3/5
63/63 [==============================] - 7s 105ms/step - loss: 0.1532 - accuracy: 0.9503 - val_loss: 0.3640 - val_accuracy: 0.8542
Epoch 4/5
63/63 [==============================] - 7s 104ms/step - loss: 0.0647 - accuracy: 0.9835 - val_loss: 0.4736 - val_accuracy: 0.8567
Epoch 5/5
63/63 [==============================] - 7s 109ms/step - loss: 0.0378 - accuracy: 0.9923 - val_loss: 0.4410 - val_accuracy: 0.8525
CPU times: user 1min 49s, sys: 34.4 s, total: 2min 23s
Wall time: 37.5 s


In [19]:
model.evaluate(test_X, test_Y)

38/38 [==============================] - 0s 12ms/step - loss: 0.3991 - accuracy: 0.8608


[0.3990863561630249, 0.8608333468437195]